In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5은 이미지 차원에 해당
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 제곱수라면, 하나의 숫자만을 특정(specify)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # 배치 차원을 제외한 모든 차원을 하나로 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to("cuda:0")
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
from torchsummary import summary

In [3]:
summary(net, (1,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Linear-3                  [-1, 120]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


/home/ubuntu/anaconda3/envs/union2/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:
for name, param in net.named_parameters():
    print(name)
    print(param)

conv1.weight
Parameter containing:
tensor([[[[-0.1141,  0.0675, -0.0598, -0.0966,  0.0142],
          [ 0.0332,  0.1022,  0.0679,  0.1704, -0.1976],
          [ 0.0868, -0.1377, -0.0313,  0.0328,  0.0511],
          [ 0.0625,  0.1513,  0.1643,  0.0087,  0.1394],
          [ 0.0284,  0.1950,  0.0518,  0.1592,  0.1713]]],


        [[[-0.0207, -0.1672, -0.0054, -0.1824,  0.1751],
          [-0.0781, -0.0989,  0.0313,  0.1176,  0.0349],
          [ 0.1820,  0.0362, -0.0944,  0.0837,  0.0907],
          [ 0.1408, -0.0040,  0.1333, -0.0228, -0.1567],
          [-0.1353, -0.0387,  0.0421,  0.0985, -0.1364]]],


        [[[ 0.0404, -0.1577, -0.1612,  0.1658, -0.1927],
          [ 0.1528,  0.1272,  0.1300,  0.0050, -0.0443],
          [-0.0274,  0.0122,  0.0415, -0.0888,  0.1035],
          [ 0.1714, -0.1849, -0.1467, -0.1021, -0.1579],
          [-0.1799, -0.1977, -0.1802, -0.0789,  0.0013]]],


        [[[-0.0997, -0.0656,  0.1030, -0.0037,  0.1064],
          [ 0.1292, -0.1470,  0.1088,  0.

In [5]:
net.state_dict()["fc3.weight"][0][0].dtype

torch.float32

In [6]:
def cut_node(torch_layer, percent):
    #정렬된 weight(절댓값 기준)
    abs_sort_result = torch.abs(torch_layer).view(-1).sort()
    cut_off_index = int(len(abs_sort_result.values) * percent)
    cut_off_value = abs_sort_result.values[cut_off_index-1]
    return torch.where(torch.abs(torch_layer) > cut_off_value, torch_layer, torch.zeros(torch_layer.size(),dtype=torch.float32).cuda())

In [7]:
percent = 0.8
for name, param in net.named_parameters():
    if "weight" in name:       
        cut = cut_node(param,percent)
        net.state_dict()[name].data.copy_(cut)



In [8]:
# 1. layer 별로 파라미터 수 count
# 2. count 된 params를 일정 percent대로 역치 찾기
# 3. threshold 보다 작은값 0으로 바꿔주기.

In [9]:
net.state_dict()["conv2.weight"].size()

torch.Size([16, 6, 5, 5])

In [10]:
type(net.state_dict()["conv2.weight"][0][0][0][1])

torch.Tensor

In [11]:
type(net.state_dict()["conv1.weight"][0][0][0][1])

torch.Tensor

In [12]:
for name, param in net.named_parameters():
    print(name)
    print(param)

conv1.weight
Parameter containing:
tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.1704, -0.1976],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.1643,  0.0000,  0.0000],
          [ 0.0000,  0.1950,  0.0000,  0.1592,  0.1713]]],


        [[[ 0.0000, -0.1672,  0.0000, -0.1824,  0.1751],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.1820,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000, -0.1577, -0.1612,  0.1658, -0.1927],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.1714, -0.1849,  0.0000,  0.0000, -0.1579],
          [-0.1799, -0.1977, -0.1802,  0.0000,  0.0000]]],


        [[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.

In [13]:
net.state_dict()["conv1.weight"][0][0][0][1].dtype

torch.float32

In [14]:
net.state_dict()["conv2.weight"][0][0][0][0]

tensor(0.0780, device='cuda:0')

In [15]:
example1 = torch.tensor([[1,2,3,4],[5,6,7,8]])

In [16]:
print(example1.nelement())

8


In [17]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(net.conv1.weight == 0))
        / float(net.conv1.weight.nelement())
    )
)

Sparsity in conv1.weight: 80.00%


In [18]:
torch.__version__

'1.9.0+cu111'

In [19]:
!conda list union2

# packages in environment at /home/ubuntu/anaconda3/envs/union2:
#
# Name                    Version                   Build  Channel


In [20]:
import sys
print(sys.version)

3.8.11 (default, Aug  3 2021, 15:09:35) 
[GCC 7.5.0]


In [21]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
